In [4]:
import os
import pandas as pd
import re

# ================== 请修改这行 ==================
folder_path = r"C:\Users\Admin\Desktop\张思远2025发票\全部发票"  # 例如：r"D:\发票文件夹"
output_excel = r"C:\Users\Admin\Desktop\张思远2025发票\张思远2025发票金额统计.xlsx"      # 输出Excel文件名（可改）
# ===============================================

data = []
for filename in os.listdir(folder_path):
    if filename.lower().endswith('.pdf'):
        name_no_ext = filename[:-4]  # 移除 .pdf
        
        # 提取末尾的金额（支持 123 或 123.45）
        match = re.search(r'(\d+\.?\d*)$', name_no_ext)
        amount = float(match.group(1)) if match else 0.0
        
        # 名称是去除金额的部分（并清理尾部多余字符）
        name_clean = re.sub(r'\d+\.?\d*$', '', name_no_ext).rstrip(' _-')
        
        data.append({
            '文件名': filename,
            '名称': name_clean,
            '金额': amount
        })

# 转成DataFrame并排序
df = pd.DataFrame(data)
df = df.sort_values('金额', ascending=False).reset_index(drop=True)

# 保存到Excel
df.to_excel(output_excel, index=False, engine='openpyxl')

print(f"完成！处理了 {len(df)} 个文件，已保存到 {os.path.abspath(output_excel)}")

完成！处理了 83 个文件，已保存到 C:\Users\Admin\Desktop\张思远2025发票\张思远2025发票金额统计.xlsx


In [5]:
import pandas as pd

# ================== 请修改这些路径 ==================
all_invoices_path = r"C:\Users\Admin\Desktop\张思远2025发票\张思远2025发票金额统计.xlsx"  # 例如：r"D:\发票\发票金额汇总.xlsx"
personal_invoices_path = r"c:\Users\Admin\Desktop\张思远2025发票\张思远2025发票金额统计-未报销.xlsx"  # 例如：r"D:\发票\个人垫付.xlsx"
output_non_personal = r"c:\Users\Admin\Desktop\张思远2025发票\非个人垫付发票.xlsx"  # 输出新Excel的路径
# =================================================

# 读取两个Excel文件
df_all = pd.read_excel(all_invoices_path)
df_personal = pd.read_excel(personal_invoices_path)

# 假设两张表都有 '文件名' 列作为唯一标识符
# 找出全部中不包含在个人垫付中的行（基于文件名匹配）
non_personal = df_all[~df_all['文件名'].isin(df_personal['文件名'])]

# 按金额从高到低排序（可选）
non_personal = non_personal.sort_values('金额', ascending=False).reset_index(drop=True)

# 保存到新Excel
non_personal.to_excel(output_non_personal, index=False, engine='openpyxl')

print(f"完成！非个人垫付发票共 {len(non_personal)} 条，已保存到 {output_non_personal}")

完成！非个人垫付发票共 49 条，已保存到 c:\Users\Admin\Desktop\张思远2025发票\非个人垫付发票.xlsx


In [7]:
import os
import pandas as pd
import pdfplumber
import re

# ================== 请修改这些路径 ==================
all_pdf_folder = r"C:\Users\Admin\Desktop\张思远2025发票\全部发票"  # 例如：r"D:\发票\所有发票"
# 注意：假设所有PDF都在这个文件夹里，包括个人和课题组的。如果个人垫付PDF在单独文件夹，可以额外处理，但这里假设都在all_pdf_folder
all_excel_path = r"C:\Users\Admin\Desktop\张思远2025发票\张思远2025发票金额统计.xlsx"  # 例如：r"D:\发票\所有发票汇总.xlsx"
personal_excel_path = r"C:\Users\Admin\Desktop\张思远2025发票\张思远2025发票金额统计-未报销.xlsx"
group_excel_path = r"C:\Users\Admin\Desktop\张思远2025发票\非个人垫付发票.xlsx"  # 非个人垫付的那个
output_excel = r"C:\Users\Admin\Desktop\张思远2025发票\发票详细信息汇总.xlsx"  # 输出文件路径
# =================================================

# 读取三个Excel，获取文件名集合（假设Excel有 '文件名' 列）
df_all = pd.read_excel(all_excel_path)
df_personal = pd.read_excel(personal_excel_path)
df_group = pd.read_excel(group_excel_path)

all_files = set(df_all['文件名'])
personal_files = set(df_personal['文件名'])
group_files = set(df_group['文件名'])

# 数据列表
data = []

# 遍历所有发票文件名
for filename in all_files:
    if not filename.lower().endswith('.pdf'):
        continue
    
    pdf_path = os.path.join(all_pdf_folder, filename)
    if not os.path.exists(pdf_path):
        print(f"警告：找不到PDF文件 {pdf_path}，跳过。")
        continue
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            # 假设发票是单页或第一页为主
            page = pdf.pages[0]
            text = page.extract_text()
            
            # 尝试提取表格（发票品名等）
            tables = page.extract_tables()
            pinming_list = []
            if tables:
                # 假设第一个表格是品名表，列0是品名，跳过表头
                item_table = tables[0]  # 可能需调整如果有多个表
                for row in item_table[1:]:  # 跳过表头
                    if row and row[0].strip():  # 品名在第0列
                        pinming = row[0].strip().replace('*', '')  # 清理*号
                        pinming_list.append(pinming)
            pinming = '；'.join(pinming_list) if pinming_list else "未知品名"
            
            # 提取发票号：发票代码: \d+
            invoice_num_match = re.search(r'发票代码[:： ]*(\d+)', text)
            invoice_num = invoice_num_match.group(1) if invoice_num_match else "未知"
            
            # 提取发票日期：开票日期: \d{4}/\d{2}/\d{2}
            invoice_date_match = re.search(r'开票日期[:： ]*(\d{4}/\d{2}/\d{2})', text)
            invoice_date = invoice_date_match.group(1) if invoice_date_match else "未知"
            
            # 提取发票金额：合计金额 ¥?([\d\.]+)
            amount_match = re.search(r'合计金额 ¥?([\d.,]+)', text)
            amount_str = amount_match.group(1).replace(',', '') if amount_match else "0.0"
            try:
                amount = float(amount_str)
            except ValueError:
                amount = 0.0
            
            # 提取供货单位：销售方名称: (.*)
            gonghuo_match = re.search(r'销售方名称[:： ]*(.*)', text)
            gonghuo = gonghuo_match.group(1).strip() if gonghuo_match else "未知"
        
        # 确定是否课题组垫付
        is_group_pad = '是' if filename in group_files else '否'
        
        # 付款方式
        pay_method = '垫付' if filename in group_files else '自付' if filename in personal_files else '未知'
        
        # 固定字段
        submit_date = '20251126'
        category = '材料费'
        inv_type = '电子'
        handler = '张思远'
        
        # 添加到数据
        data.append({
            '是否由课题组垫付': is_group_pad,
            '提交日期': submit_date,
            '发票号': invoice_num,
            '发票日期': invoice_date,
            '发票金额': amount,
            '类别': category,
            '品名': pinming,
            '供货单位': gonghuo,
            '发票类型': inv_type,
            '付款方式': pay_method,
            '经办人': handler
        })
        
        print(f"处理完成: {filename}")
    
    except Exception as e:
        print(f"错误处理 {filename}: {str(e)}")

# 转为DataFrame并保存
if data:
    df_summary = pd.DataFrame(data)
    df_summary.to_excel(output_excel, index=False, engine='openpyxl')
    print(f"完成！汇总了 {len(data)} 张发票，已保存到 {os.path.abspath(output_excel)}")
else:
    print("没有处理任何发票，请检查路径和文件。")

处理完成: 有色金属冶炼压延品铜351.pdf
处理完成: 蠕动泵98.90.pdf
处理完成: 显微镜投146.69.pdf
处理完成: 磁铁21.pdf
处理完成: 高纯铜片153.pdf
处理完成: 万用表72.36.pdf
处理完成: 磁铁4.9.pdf
处理完成: 磁铁6.7.pdf
处理完成: 导轨3.53.pdf
处理完成: USB转113.95.pdf
处理完成: usb拓展518.pdf
处理完成: 485集线器125.pdf
处理完成: 显示器连接线39.pdf
处理完成: 螺柱19.5.pdf
处理完成: 倒锥接头特氟龙管512.pdf
处理完成: 电极402.2.pdf
处理完成: 石英玻璃管150.pdf
处理完成: 3d打印耗材941.45.pdf
处理完成: 显微相机850.pdf
处理完成: 茄形瓶10.84.pdf
处理完成: 魔术贴理线器18.81.pdf
处理完成: 氟橡胶管79.6.pdf
处理完成: 氟橡胶管60.pdf
处理完成: 磁铁20.pdf
处理完成: 接线端子55.8.pdf
处理完成: 摄像头65.pdf
处理完成: 铝型材930.pdf
处理完成: US转119.pdf
处理完成: 橡胶管3.49.pdf
处理完成: 线槽14.6.pdf
处理完成: 磁铁23.pdf
处理完成: 强力胶带7.92.pdf
处理完成: 端子线7.4.pdf
处理完成: 鳄鱼夹37.4.pdf
处理完成: 磁铁57.60.pdf
处理完成: 倒锥接头241.05.pdf
处理完成: 电机套装203.41.pdf
处理完成: 试剂瓶418.93.pdf
处理完成: 剪管器15.04.pdf
处理完成: 砂纸15.3.pdf
处理完成: 分线器73.4.pdf
处理完成: 倒锥接头912.pdf
处理完成: 电线31.pdf
处理完成: 磁吸壁灯216.13.pdf
处理完成: 铝型材配件69.pdf
处理完成: 电源96.96.pdf
处理完成: 滑台532.pdf
处理完成: 插座79.8.pdf
处理完成: 电机86.99.pdf
处理完成: 收纳箱131.57.pdf
处理完成: 扎带3.8.pdf
处理完成: 剪刀23.80.pdf
处理完成: 3M强力布基胶带11.37.pdf
处理完成: 解胶剂10.5.pdf
处理完

In [9]:
import os
import pandas as pd
import pdfplumber
import re

# ================== 请修改这些路径 ==================
all_pdf_folder = r"C:\Users\Admin\Desktop\张思远2025发票\全部发票"  # 例如：r"D:\发票\所有发票"
# 注意：假设所有PDF都在这个文件夹里，包括个人和课题组的。如果个人垫付PDF在单独文件夹，可以额外处理，但这里假设都在all_pdf_folder
all_excel_path = r"C:\Users\Admin\Desktop\张思远2025发票\张思远2025发票金额统计.xlsx"  # 例如：r"D:\发票\所有发票汇总.xlsx"
personal_excel_path = r"C:\Users\Admin\Desktop\张思远2025发票\张思远2025发票金额统计-未报销.xlsx"
group_excel_path = r"C:\Users\Admin\Desktop\张思远2025发票\非个人垫付发票.xlsx"  # 非个人垫付的那个
output_excel = r"C:\Users\Admin\Desktop\张思远2025发票\发票详细信息汇总.xlsx"  # 输出文件路径
# =================================================

# 读取Excel获取文件名集合
df_all = pd.read_excel(all_excel_path)
df_personal = pd.read_excel(personal_excel_path)
df_group = pd.read_excel(group_excel_path)

all_files = set(df_all['文件名'])
personal_files = set(df_personal['文件名'])
group_files = set(df_group['文件名'])

data = []

for filename in all_files:
    if not filename.lower().endswith('.pdf'):
        continue
    
    pdf_path = os.path.join(all_pdf_folder, filename)
    if not os.path.exists(pdf_path):
        print(f"警告：找不到 {pdf_path}")
        continue
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            page = pdf.pages[0]
            text = page.extract_text() or ""
            if not text:
                print(f"警告：{filename} 无文本")
                continue
            
            # 调试：打印文本查看提取
            # print(f"文本 for {filename}:\n{text}\n")
            
            # 发票号：更鲁棒正则，支持空格、全角
            invoice_num_match = re.search(r'发票代码\s*[:： ]\s*(\d+)', text, re.IGNORECASE)
            invoice_num = invoice_num_match.group(1) if invoice_num_match else "未知"
            
            # 发票日期
            invoice_date_match = re.search(r'开票日期\s*[:： ]\s*(\d{4}/\d{2}/\d{2})', text, re.IGNORECASE)
            invoice_date = invoice_date_match.group(1) if invoice_date_match else "未知"
            
            # 发票金额：合计金额后 ¥ 或小写金额，移除逗号
            amount_match = re.search(r'合计金额\s*¥?\s*([\d.,]+)', text, re.IGNORECASE)
            amount_str = amount_match.group(1).replace(',', '') if amount_match else "0.0"
            try:
                amount = float(amount_str)
            except ValueError:
                amount = 0.0
            
            # 供货单位
            gonghuo_match = re.search(r'销售方名称\s*[:： ]\s*([^\n]+)', text, re.IGNORECASE)
            gonghuo = gonghuo_match.group(1).strip() if gonghuo_match else "未知"
            
            # 品名：尝试提取表格
            tables = page.extract_tables()
            pinming_list = []
            for table in tables:  # 循环所有表格找品名
                for row in table:
                    if row and len(row) > 0 and ('品名' in row[0] or row[0].strip().startswith('*')):
                        pinming = row[0].strip().replace('*', '').replace('品名（规格型号）', '').strip()
                        if pinming:
                            pinming_list.append(pinming)
            if not pinming_list:  # 如果表格没找到，用正则找品名后文本
                pinming_match = re.search(r'\*品名规格型号\s*(.+)', text, re.DOTALL)
                if pinming_match:
                    pinming_list = [p.strip() for p in pinming_match.group(1).split('\n') if p.strip()]
            pinming = '；'.join(pinming_list) if pinming_list else "未知品名"
        
        # 其他字段
        is_group_pad = '是' if filename in group_files else '否'
        pay_method = '垫付' if filename in group_files else '自付' if filename in personal_files else '未知'
        submit_date = '20251126'
        category = '材料费'
        inv_type = '电子'
        handler = '张思远'
        
        data.append({
            '是否由课题组垫付': is_group_pad,
            '提交日期': submit_date,
            '发票号': invoice_num,
            '发票日期': invoice_date,
            '发票金额': amount,
            '类别': category,
            '品名': pinming,
            '供货单位': gonghuo,
            '发票类型': inv_type,
            '付款方式': pay_method,
            '经办人': handler
        })
        
        print(f"处理: {filename} - 发票号: {invoice_num}, 日期: {invoice_date}, 金额: {amount}")
    
    except Exception as e:
        print(f"错误 {filename}: {str(e)}")

if data:
    df_summary = pd.DataFrame(data)
    df_summary.to_excel(output_excel, index=False, engine='openpyxl')
    print(f"完成！汇总 {len(data)} 张，已保存 {os.path.abspath(output_excel)}")
else:
    print("无发票处理")

处理: 有色金属冶炼压延品铜351.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 蠕动泵98.90.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 显微镜投146.69.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 磁铁21.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 高纯铜片153.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 万用表72.36.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 磁铁4.9.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 磁铁6.7.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 导轨3.53.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: USB转113.95.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: usb拓展518.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 485集线器125.pdf - 发票号: 144032509110, 日期: 未知, 金额: 0.0
处理: 显示器连接线39.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 螺柱19.5.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 倒锥接头特氟龙管512.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 电极402.2.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 石英玻璃管150.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 3d打印耗材941.45.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 显微相机850.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 茄形瓶10.84.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 魔术贴理线器18.81.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 氟橡胶管79.6.pdf - 发票号: 未知, 日期: 未知, 金额: 0.0
处理: 氟橡胶管60.pdf - 发票

In [12]:
import os
import pandas as pd
import pdfplumber
import re

# ================== 路径设置 ==================
all_pdf_folder = r"C:\Users\Admin\Desktop\张思远2025发票\全部发票"
all_excel_path = r"C:\Users\Admin\Desktop\张思远2025发票\张思远2025发票金额统计.xlsx"
personal_excel_path = r"C:\Users\Admin\Desktop\张思远2025发票\张思远2025发票金额统计-未报销.xlsx"
group_excel_path = r"C:\Users\Admin\Desktop\张思远2025发票\非个人垫付发票.xlsx"
output_excel = r"C:\Users\Admin\Desktop\张思远2025发票\发票详细信息汇总.xlsx"
# ============================================

# 读取Excel列表（保持不变）
try:
    df_all = pd.read_excel(all_excel_path)
    df_personal = pd.read_excel(personal_excel_path)
    df_group = pd.read_excel(group_excel_path)
    
    all_files = set(df_all['文件名'].astype(str).str.strip())
    personal_files = set(df_personal['文件名'].astype(str).str.strip())
    group_files = set(df_group['文件名'].astype(str).str.strip())
except Exception as e:
    print(f"读取Excel失败: {e}")
    exit()

data = []

def clean_text(text):
    if not text: return ""
    return text.replace(" ", "").replace("\n", "")

print("开始处理发票...")

for filename in os.listdir(all_pdf_folder):
    if not filename.lower().endswith('.pdf'):
        continue

    pdf_path = os.path.join(all_pdf_folder, filename)
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            page = pdf.pages[0]
            width = page.width
            height = page.height
            full_text = clean_text(page.extract_text())

            # === 1. 修复：供货单位（去噪） ===
            # 切割右侧区域，找“名称”
            seller_box = (width * 0.5, height * 0.1, width, height * 0.5)
            seller_text = clean_text(page.crop(seller_box).extract_text())
            
            # 正则匹配名称
            # 解释：匹配“名称”或“称”开头，提取中文/英文/括号，直到遇到“纳税”、“销售”、“售方”等字样停止
            seller_match = re.search(r'名称[:：]?([\u4e00-\u9fa5A-Za-z0-9（）()]+)', seller_text)
            
            gonghuo = "提取失败"
            if seller_match:
                raw_name = seller_match.group(1)
                # 【关键修复】去掉末尾的“售方信”、“销售方”、“纳税”等杂质
                gonghuo = re.split(r'售方|销售|纳税|密码|地址', raw_name)[0]

            # === 2. 修复：品名（找对表格） ===
            tables = page.extract_tables()
            pinming_list = []
            valid_table_found = False
            
            if tables:
                for table in tables:
                    # 将表格转为字符串粗略检查，必须包含“货物”、“名称”、“项目”或“规格”才认为是物品表
                    table_str = str(table)
                    if "货物" in table_str or "名称" in table_str or "规格" in table_str or "项目" in table_str:
                        # 找到了物品表！
                        for row in table:
                            # 跳过无效行
                            if not row or len(row) < 2: continue 
                            first_col = str(row[0]).strip()
                            
                            # 跳过表头和合计行
                            if any(x in first_col for x in ["名称", "货物", "项目", "合计", "小写", "购买方", "备注"]):
                                continue
                            
                            # 提取内容
                            clean_name = first_col.replace('\n', '')
                            # 去掉 *分类编码*
                            if '*' in clean_name:
                                parts = clean_name.split('*')
                                item_name = parts[-1] if parts[-1] else parts[-2] # 取最后一段
                            else:
                                item_name = clean_name
                            
                            if item_name:
                                pinming_list.append(item_name)
                        
                        # 只要处理了一个符合条件的表格，就认为提取成功，停止遍历其他布局表格
                        if pinming_list:
                            valid_table_found = True
                            break
            
            pinming = '；'.join(pinming_list) if pinming_list else "未识别到明细"

            # === 3. 修复：金额（优先小写，保底末尾数字） ===
            amount = 0.0
            # 策略A：找“小写”后面的数字 (最准)
            amount_match = re.search(r'小写.*?[¥￥]?([\d\.]+)', full_text)
            
            if amount_match:
                amount = float(amount_match.group(1))
            else:
                # 策略B：如果没有“小写”字样，找全篇最后一个像金额的数字
                # 匹配所有数字（包含小数或整数），排除日期格式
                # 这里的逻辑是：找出所有数字，取最后一个浮点数
                all_numbers = re.findall(r'(\d+\.\d{2})', full_text) # 找带2位小数的
                if not all_numbers:
                    all_numbers = re.findall(r'(\d+)', full_text) # 找整数
                
                if all_numbers:
                    # 取最后一个，通常是合计
                    try:
                        possible_amount = float(all_numbers[-1])
                        # 简单过滤：日期里的2025通常不会是金额，且金额通常不会特别大或特别小
                        if possible_amount != 2025 and possible_amount != 2024: 
                            amount = possible_amount
                    except:
                        pass

            # === 4. 其他信息（日期/号码） ===
            # 继续使用右上角切割法，这个通常比较稳
            header_box = (width * 0.5, 0, width, height * 0.25)
            header_text = clean_text(page.crop(header_box).extract_text())
            
            num_match = re.search(r'号码[:：]?(\d+)', header_text)
            invoice_num = num_match.group(1) if num_match else "提取失败"
            
            date_match = re.search(r'(\d{4}[年/]\d{2}[月/]\d{2})', header_text)
            if date_match:
                invoice_date = date_match.group(1).replace('年', '/').replace('月', '/').replace('日', '')
            else:
                invoice_date = "提取失败"

        # === 逻辑分类 ===
        is_group_pad = '是' if filename in group_files else '否'
        if filename in group_files:
            pay_method = '垫付'
        elif filename in personal_files:
            pay_method = '自付'
        else:
            pay_method = '已报销/其他'

        data.append({
            '文件名': filename,
            '是否由课题组垫付': is_group_pad,
            '提交日期': '20251126',
            '发票号': invoice_num,
            '发票日期': invoice_date,
            '发票金额': amount,
            '类别': '材料费',
            '品名': pinming,
            '供货单位': gonghuo,
            '发票类型': '电子',
            '付款方式': pay_method,
            '经办人': '张思远'
        })
        
        print(f"处理: {filename} -> 品名: {pinming[:15]}... | 金额: {amount} | 供货: {gonghuo}")

    except Exception as e:
        print(f"❌ 出错 {filename}: {str(e)}")

# 保存
if data:
    df_out = pd.DataFrame(data)
    cols = ['文件名', '是否由课题组垫付', '提交日期', '发票号', '发票日期', '发票金额', 
            '类别', '品名', '供货单位', '发票类型', '付款方式', '经办人']
    df_out = df_out[cols]
    df_out.to_excel(output_excel, index=False)
    print(f"\n✅ 完成！结果已保存至 {output_excel}")

开始处理发票...
处理: 3d打印耗材941.45.pdf -> 品名: 购买方信息；备注... | 金额: 941.45 | 供货: 深圳拓竹科技有限公司
处理: 3M强力布基胶带11.37.pdf -> 品名: 购买方信息；备注... | 金额: 11.37 | 供货: 深圳市卡优圣科技有限公司
处理: 485集线器125.pdf -> 品名: 购买方；销货方... | 金额: 125.0 | 供货: 提取失败
处理: usb拓展13.8.pdf -> 品名: 购买方信息；备注... | 金额: 13.8 | 供货: 珍明贸易（上海）有限公司
处理: usb拓展518.pdf -> 品名: 购买方信息；备注... | 金额: 518.0029654025267 | 供货: 武汉维联特商贸有限公司
处理: USB转113.95.pdf -> 品名: 购买方信息；备注... | 金额: 113.95 | 供货: 深圳市至来科技有限公司
处理: USB转119.pdf -> 品名: 购买方信息；备注... | 金额: 119.0 | 供货: 深圳市至来科技有限公司
处理: US转119.pdf -> 品名: 购买方信息；备注... | 金额: 119.0 | 供货: 深圳市至来科技有限公司
处理: 万用表72.36.pdf -> 品名: 购买方信息；备注... | 金额: 72.36 | 供货: 烟台市绿林五金有限公司
处理: 倒锥接头241.05.pdf -> 品名: 购买方信息；倒锥接头 WD08... | 金额: 241.05 | 供货: 南京润泽流体控制设备有限公司
处理: 倒锥接头912.pdf -> 品名: 购买方信息；备注... | 金额: 912.0 | 供货: 南京润泽流体控制设备有限公司
处理: 倒锥接头特氟龙管512.pdf -> 品名: 购买方信息；备注... | 金额: 512.0 | 供货: 南京润泽流体控制设备有限公司
处理: 倒锥接头特氟龙管787.8.pdf -> 品名: 购买方信息；备注... | 金额: 787.8 | 供货: 南京润泽流体控制设备有限公司
处理: 分线器73.4.pdf -> 品名: 购买方信息；备注... | 金额: 73.40295859100027 | 供货: 温州元璋电气有限公司
处理: 剥线钳压线钳7

In [14]:
import os
import pandas as pd
import pdfplumber
import re

# ================== 路径设置 ==================
all_pdf_folder = r"C:\Users\Admin\Desktop\张思远2025发票\全部发票"
output_excel = r"C:\Users\Admin\Desktop\张思远2025发票\发票详细信息汇总3.xlsx"
# ============================================

data = []

def clean_text(text):
    """只去除换行，保留空格和特殊符号"""
    if not text: return ""
    return text.replace("\n", "")

print("开始处理...")

for filename in os.listdir(all_pdf_folder):
    if not filename.lower().endswith('.pdf'):
        continue

    pdf_path = os.path.join(all_pdf_folder, filename)
    
    # === 1. 金额：直接从文件名提取 (按您要求的) ===
    amount = 0.0
    try:
        # 找文件名里 .pdf 前面的数字，或者最后一个数字
        # 例如: "3打印耗材941.45.pdf" -> 941.45
        amount_match = re.findall(r'(\d+\.?\d*)', filename)
        if amount_match:
            # 取最后一个匹配到的数字（通常文件名末尾的是金额）
            # 过滤掉 purely '.pdf' 产生的空结果
            valid_nums = [float(x) for x in amount_match if x != '.' and x != '']
            if valid_nums:
                amount = valid_nums[-1]
    except:
        amount = 0.0

    try:
        with pdfplumber.open(pdf_path) as pdf:
            page = pdf.pages[0]
            width = page.width
            height = page.height

            # === 2. 品名：粗暴提取“项目名称”列的所有内容 ===
            tables = page.extract_tables()
            pinming_list = []
            
            if tables:
                for table in tables:
                    if not table or not table[0]: continue
                    
                    # 检查表头（第一行），确认这是物品清单表
                    # 只要表头包含“名称”且包含“金额”或“单价”，就是它了
                    header_str = "".join([str(x) for x in table[0] if x])
                    
                    if ("名称" in header_str or "货物" in header_str) and ("金额" in header_str or "单价" in header_str or "数量" in header_str):
                        
                        # 遍历该表格的所有行（跳过第1行表头）
                        for row in table[1:]:
                            if not row: continue
                            
                            # 取第1列（项目名称列）
                            cell_text = str(row[0]).strip()
                            
                            # 遇到“合计”、“小写”、“备注”行就停止，防止读到底部杂讯
                            if "合计" in cell_text or "小写" in cell_text or "备注" in cell_text:
                                continue
                            
                            # 只要内容不为空，就加进去！不做任何切割！
                            if cell_text:
                                # 去掉换行符即可
                                full_content = cell_text.replace('\n', '')
                                pinming_list.append(full_content)
            
            # 用分号连接多行品名
            pinming = '；'.join(pinming_list) if pinming_list else "未提取到"

            # === 3. 供货单位：右侧区域截取 ===
            seller_box = (width * 0.5, height * 0.08, width, height * 0.45)
            seller_text = clean_text(page.crop(seller_box).extract_text())
            
            # 提取“名称”后的内容
            seller_match = re.search(r'名称[:：]?([\u4e00-\u9fa5A-Za-z0-9（）\(\)]+)', seller_text)
            gonghuo = "提取失败"
            if seller_match:
                temp = seller_match.group(1)
                # 截断杂质
                stop_words = ['销售', '纳税', '统一', '地址', '开户', '密码', '售方']
                for word in stop_words:
                    if word in temp:
                        temp = temp.split(word)[0]
                gonghuo = temp

            # === 4. 基础信息 ===
            header_box = (width * 0.5, 0, width, height * 0.25)
            header_text = clean_text(page.crop(header_box).extract_text())
            
            num_match = re.search(r'号码[:：]?(\d+)', header_text)
            invoice_num = num_match.group(1) if num_match else "提取失败"
            
            date_match = re.search(r'(\d{4}[年/]\d{2}[月/]\d{2})', header_text)
            if date_match:
                invoice_date = date_match.group(1).replace('年', '/').replace('月', '/').replace('日', '')
            else:
                invoice_date = "提取失败"

        # 添加数据
        data.append({
            '文件名': filename,
            '发票号': invoice_num,
            '发票日期': invoice_date,
            '发票金额': amount,
            '品名': pinming,
            '供货单位': gonghuo,
            '类别': '材料费',
            '经办人': '张思远'
        })
        
        print(f"ok: {filename} -> 品名: {pinming} | 供货: {gonghuo}")

    except Exception as e:
        print(f"error: {filename} -> {e}")

# 保存
if data:
    df = pd.DataFrame(data)
    cols = ['文件名', '发票号', '发票日期', '发票金额', '品名', '供货单位', '类别', '经办人']
    df = df[[c for c in cols if c in df.columns]]
    df.to_excel(output_excel, index=False)
    print(f"\n完成！已保存到 {output_excel}")

开始处理...
ok: 3d打印耗材941.45.pdf -> 品名: 未提取到 | 供货: 深圳拓竹科技有限公司
ok: 3M强力布基胶带11.37.pdf -> 品名: 未提取到 | 供货: 深圳市卡优圣科技有限公司
ok: 485集线器125.pdf -> 品名: 未提取到 | 供货: 提取失败
ok: usb拓展13.8.pdf -> 品名: 未提取到 | 供货: 珍明贸易（上海）有限公司
ok: usb拓展518.pdf -> 品名: 未提取到 | 供货: 武汉维联特商贸有限公司
ok: USB转113.95.pdf -> 品名: 未提取到 | 供货: 深圳市至来科技有限公司
ok: USB转119.pdf -> 品名: 未提取到 | 供货: 深圳市至来科技有限公司
ok: US转119.pdf -> 品名: 未提取到 | 供货: 深圳市至来科技有限公司
ok: 万用表72.36.pdf -> 品名: 未提取到 | 供货: 烟台市绿林五金有限公司
ok: 倒锥接头241.05.pdf -> 品名: 未提取到 | 供货: 提取失败
ok: 倒锥接头912.pdf -> 品名: 未提取到 | 供货: 南京润泽流体控制设备有限公司
ok: 倒锥接头特氟龙管512.pdf -> 品名: 未提取到 | 供货: 南京润泽流体控制设备有限公司
ok: 倒锥接头特氟龙管787.8.pdf -> 品名: 未提取到 | 供货: 南京润泽流体控制设备有限公司
ok: 分线器73.4.pdf -> 品名: 未提取到 | 供货: 温州元璋电气有限公司
ok: 剥线钳压线钳79.52.pdf -> 品名: 未提取到 | 供货: 烟台市绿林五金有限公司
ok: 剪刀23.80.pdf -> 品名: 未提取到 | 供货: 临沂平通商贸有限公司
ok: 剪管器15.04.pdf -> 品名: 未提取到 | 供货: 南京润泽流体控制设备有限公司
ok: 双面胶7.84.pdf -> 品名: 未提取到 | 供货: 方信
ok: 导电胶带4.87.pdf -> 品名: 未提取到 | 供货: 方信
ok: 导轨3.53.pdf -> 品名: 未提取到 | 供货: 乐清市众畅电器厂
ok: 强力胶带7.92.pdf -> 品名: 未提取到 | 供货: 深圳市奔亿达科技有限公司
ok: 扎带3.8.p

In [15]:
import os
import re
import pandas as pd
import pdfplumber
from pathlib import Path

def extract_invoice_info(pdf_path):
    """
    从PDF发票中提取关键信息
    """
    try:
        with pdfplumber.open(pdf_path) as pdf:
            # 读取所有页面的文本
            text = ''
            for page in pdf.pages:
                text += page.extract_text() + '\n'
            
            # 提取发票号码
            invoice_no_match = re.search(r'发票号码[：:]\s*(\d+)', text)
            invoice_no = invoice_no_match.group(1) if invoice_no_match else ''
            
            # 提取开票日期
            date_match = re.search(r'开票日期[：:]\s*(\d{4}年\d{1,2}月\d{1,2}日)', text)
            invoice_date = date_match.group(1) if date_match else ''
            
            # 提取销售方名称(供货单位)
            # 先找到"销售方信息"区域,然后在其后找"名称"
            supplier_match = re.search(r'销[\s\S]{0,100}名称[：:]\s*([^\s\n]+)', text)
            supplier = supplier_match.group(1) if supplier_match else ''
            
            # 提取购买方名称
            buyer_match = re.search(r'购[\s\S]{0,100}名称[：:]\s*([^\s\n]+)', text)
            buyer = buyer_match.group(1) if buyer_match else ''
            
            # 提取销售方税号
            supplier_tax_match = re.search(r'销[\s\S]{0,150}统一社会信用代码/纳税人识别号[：:]\s*([A-Z0-9]+)', text)
            supplier_tax_no = supplier_tax_match.group(1) if supplier_tax_match else ''
            
            # 提取购买方税号
            buyer_tax_match = re.search(r'购[\s\S]{0,150}统一社会信用代码/纳税人识别号[：:]\s*([A-Z0-9]+)', text)
            buyer_tax_no = buyer_tax_match.group(1) if buyer_tax_match else ''
            
            # 提取项目名称(品名) - 提取所有*号之间的内容
            items = re.findall(r'\*([^*]+)\*', text)
            items_str = '; '.join([item.strip() for item in items if item.strip()])
            
            # 提取价税合计金额
            amount_match = re.search(r'价税合计[^¥]*¥([\d,]+\.?\d*)', text)
            total_amount = amount_match.group(1) if amount_match else ''
            
            # 提取税额
            tax_match = re.search(r'合\s*计[^¥]*¥[\d,]+\.?\d*[^¥]*¥([\d,]+\.?\d*)', text)
            tax_amount = tax_match.group(1) if tax_match else ''
            
            # 提取不含税金额
            amount_no_tax_match = re.search(r'合\s*计[^¥]*¥([\d,]+\.?\d*)', text)
            amount_no_tax = amount_no_tax_match.group(1) if amount_no_tax_match else ''
            
            return {
                '文件名': os.path.basename(pdf_path),
                '发票号码': invoice_no,
                '开票日期': invoice_date,
                '供货单位': supplier,
                '供货单位税号': supplier_tax_no,
                '购买方': buyer,
                '购买方税号': buyer_tax_no,
                '品名': items_str,
                '不含税金额': amount_no_tax,
                '税额': tax_amount,
                '价税合计': total_amount,
                '状态': '成功'
            }
            
    except Exception as e:
        return {
            '文件名': os.path.basename(pdf_path),
            '发票号码': '',
            '开票日期': '',
            '供货单位': '',
            '供货单位税号': '',
            '购买方': '',
            '购买方税号': '',
            '品名': '',
            '不含税金额': '',
            '税额': '',
            '价税合计': '',
            '状态': f'失败: {str(e)}'
        }

def process_pdf_folder(folder_path, output_excel='发票汇总.xlsx'):
    """
    处理文件夹中的所有PDF发票
    """
    # 获取文件夹中所有PDF文件
    pdf_files = list(Path(folder_path).glob('*.pdf'))
    
    if not pdf_files:
        print(f"在 {folder_path} 中没有找到PDF文件")
        return
    
    print(f"找到 {len(pdf_files)} 个PDF文件，开始处理...")
    
    # 提取所有发票信息
    results = []
    for i, pdf_file in enumerate(pdf_files, 1):
        print(f"正在处理 ({i}/{len(pdf_files)}): {pdf_file.name}")
        info = extract_invoice_info(str(pdf_file))
        results.append(info)
    
    # 转换为DataFrame并保存为Excel
    df = pd.DataFrame(results)
    
    # 保存到Excel
    output_path = os.path.join(folder_path, output_excel)
    df.to_excel(output_path, index=False, engine='openpyxl')
    
    print(f"\n处理完成!")
    print(f"成功: {len([r for r in results if r['状态'] == '成功'])} 个")
    print(f"失败: {len([r for r in results if r['状态'] != '成功'])} 个")
    print(f"结果已保存到: {output_path}")
    
    return df

# 主程序
if __name__ == "__main__":
    # 设置PDF文件所在的文件夹路径
    folder_path = r"C:\Users\Admin\Desktop\张思远2025发票\全部发票"  # 修改为你的文件夹路径
    
    # 如果想使用当前脚本所在目录，取消下面这行的注释
    # folder_path = os.path.dirname(os.path.abspath(__file__))
    
    print("="*50)
    print("PDF发票信息提取工具")
    print("="*50)
    print(f"目标文件夹: {folder_path}\n")
    
    # 处理PDF文件
    df = process_pdf_folder(folder_path)
    
    # 显示前5条结果预览
    if df is not None and not df.empty:
        print("\n前5条结果预览:")
        print(df.head().to_string())

PDF发票信息提取工具
目标文件夹: C:\Users\Admin\Desktop\张思远2025发票\全部发票

找到 83 个PDF文件，开始处理...
正在处理 (1/83): 3d打印耗材941.45.pdf
正在处理 (2/83): 3M强力布基胶带11.37.pdf
正在处理 (3/83): 485集线器125.pdf
正在处理 (4/83): usb拓展13.8.pdf
正在处理 (5/83): usb拓展518.pdf
正在处理 (6/83): USB转113.95.pdf
正在处理 (7/83): USB转119.pdf
正在处理 (8/83): US转119.pdf
正在处理 (9/83): 万用表72.36.pdf
正在处理 (10/83): 倒锥接头241.05.pdf
正在处理 (11/83): 倒锥接头912.pdf
正在处理 (12/83): 倒锥接头特氟龙管512.pdf
正在处理 (13/83): 倒锥接头特氟龙管787.8.pdf
正在处理 (14/83): 分线器73.4.pdf
正在处理 (15/83): 剥线钳压线钳79.52.pdf
正在处理 (16/83): 剪刀23.80.pdf
正在处理 (17/83): 剪管器15.04.pdf
正在处理 (18/83): 双面胶7.84.pdf
正在处理 (19/83): 导电胶带4.87.pdf
正在处理 (20/83): 导轨3.53.pdf
正在处理 (21/83): 强力胶带7.92.pdf
正在处理 (22/83): 扎带3.8.pdf
正在处理 (23/83): 扎带9.7.pdf
正在处理 (24/83): 接线端子55.8.pdf
正在处理 (25/83): 插座79.8.pdf
正在处理 (26/83): 摄像头65.pdf
正在处理 (27/83): 收纳柜127.pdf
正在处理 (28/83): 收纳箱131.57.pdf
正在处理 (29/83): 无线网卡53.9.pdf
正在处理 (30/83): 显微相机850.pdf
正在处理 (31/83): 显微镜投146.69.pdf
正在处理 (32/83): 显示器连接线39.pdf
正在处理 (33/83): 有色金属冶炼压延品铜351.pdf
正在处理 (34/83): 桌子70.16.pdf
正在